In [1]:
!pip install pettingzoo[mpe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 816 kB 6.3 MB/s 
     |████████████████████████████████| 836 kB 50.3 MB/s 
     |████████████████████████████████| 18.3 MB 93 kB/s 


In [2]:
import os
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense

class ActorCriticNetwork(keras.Model):
  def __init__(self,n_actions, dims_1 = 256 , dims_2 = 256,
               name = 'actor_critc', chkpt_dir = 'tmp/actor_critic'):
    super(ActorCriticNetwork, self).__init__()
    self.dims_1 = dims_1
    self.dims_2 = dims_2
    self.n_actions = n_actions
    self.model_name = name
    self.checkpoint_dir = chkpt_dir
    self.checkpoint_file = os.path.join(self.checkpoint_dir, name+'_ac')

    self.fc1 = Dense(self.dims_1,activation = 'relu')
    self.fc2 = Dense(self.dims_2,activation = 'relu')
    self.v = Dense(1,activation = None)
    self.po = Dense(n_actions,activation = 'softmax')

  def call(self,state):
    value = self.fc1(state)
    value = self.fc2(value)

    v = self.v(value)
    po = self.po(value)

    return v,po

In [3]:
from re import A
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp


class Agent:
  def __init__(self, alpha = 1e-5, gamma = 0.95, n_actions = 4,name='agent'):
    self.gamma = gamma
    self.n_actions = n_actions
    self.action = None
    self.action_space = [i for i in range(self.n_actions)]
    self.name = name
    self.actor_critic = ActorCriticNetwork(n_actions=n_actions)
    self.actor_critic.compile(optimizer = Adam(learning_rate = alpha))
    self.fname = 'actor_critic_'+self.name

   
    self.reward_history = []
    self.action_history = []
    self.obsv_history = []
    self.next_obsv_history = []

  def choose_action(self,observation):
    state = tf.convert_to_tensor([observation])
    _,probs = self.actor_critic(state)

    action_probs = tfp.distributions.Categorical(probs = probs)
    action = action_probs.sample()

    self.action = action

    return action.numpy()[0]

  def save_models(self):
    print('saving model.....')
    self.actor_critic.save(self.fname)
  
  def load_models(self):
    #print('loading model....')
    self.actor_critic = keras.models.load_model(self.fname)

  def store_trans(self,action,reward,obsv,obsv_):
      self.action_history.append(action)
      self.reward_history.append(reward)
      self.obsv_history.append(obsv)
      self.next_obsv_history.append(obsv_)

  def clear_memory(self):
    self.reward_history = []
    self.action_history = []
    self.obsv_history = []
    self.next_obsv_history = []

  def learn(self,state,reward,state_,done):
    state = tf.convert_to_tensor([state],dtype = tf.float32)
    state_ = tf.convert_to_tensor([state_],dtype = tf.float32)
    reward = tf.convert_to_tensor(reward,dtype = tf.float32)

    with tf.GradientTape() as tape:
      state_value, probs = self.actor_critic(state)
      state_value_,_ = self.actor_critic(state_)
      state_value = tf.squeeze(state_value)
      state_value_ = tf.squeeze(state_value_)

      action_probs = tfp.distributions.Categorical(probs = probs)
      log_probs = action_probs.log_prob(self.action)

      delta = reward + self.gamma*state_value_*(1-int(done)) - state_value

      actor_loss = -log_probs*delta
      critic_loss = delta**2

      total_loss = actor_loss + critic_loss

    gradients = tape.gradient(total_loss,self.actor_critic.trainable_variables)
    self.actor_critic.optimizer.apply_gradients(zip(gradients, self.actor_critic.trainable_variables))

In [4]:
from pettingzoo.mpe import simple_speaker_listener_v3

env = simple_speaker_listener_v3.env(max_cycles = 25 , continuous_actions = False)
env.reset()
agent_list = []
for a in env.agents:
  agent_list.append(a)

In [5]:
env.reset()
scores = []
agent_score = {}
hscore = -1000
agent_high_scores = {}
policy_net = {}
for a in env.agents:
  act_n = env.action_space(a).n
  agent = Agent(alpha = 1e-5,n_actions = act_n,name = a)
  agent_score[a] = scores
  agent_high_scores[a] = hscore
  policy_net[a] = agent

In [ ]:
import numpy as np
avg_scores= []
n_games = 1000
best_score = -10000
score_history = []
load_checkpoint = False
prev_states = []
for i in range(n_games):
  env.reset()
  score = 0
  for agent in env.agent_iter():
    obsv,reward,done,trunc,_ = env.last()
    prev_states.append(obsv)
    if not (done or trunc):
      action = policy_net[agent].choose_action(obsv)
      score += reward
      env.step(action)
      obsv_,_,_,_,_ = env.last()
    else:
      action = None
      env.step(action)
    policy_net[agent].learn(obsv,reward,prev_states[-1],done)
    
  score_history.append(score)

  avg_score = np.mean(score_history[-100:])
  
  if avg_score > best_score:
    best_score = avg_score
    for a in agent_list:
      policy_net[a].save_models()

        
  print('episode: ',i ,'soore: ',score,' avg_score: ', avg_score)
  avg_scores.append(avg_score)

saving model.....
saving model.....
episode:  0 soore:  -91.00413583100885  avg_score:  -91.00413583100885
saving model.....
saving model.....
episode:  1 soore:  -13.136513809246434  avg_score:  -52.070324820127645
episode:  2 soore:  -119.86820980817767  avg_score:  -74.66961981614432
episode:  3 soore:  -183.58894637078802  avg_score:  -101.89945145480525
episode:  4 soore:  -141.6664039377901  avg_score:  -109.85284195140221
episode:  5 soore:  -68.17194776948688  avg_score:  -102.90602625441632
episode:  6 soore:  -146.6885247684609  avg_score:  -109.16066889927983
episode:  7 soore:  -126.46145934777263  avg_score:  -111.32326770534144
episode:  8 soore:  -95.66017536095521  avg_score:  -109.58292411152075
episode:  9 soore:  -1.529872958779796  avg_score:  -98.77761899624666
episode:  10 soore:  -40.930144855347066  avg_score:  -93.51875771071032


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

x = np.linspace(0,2*np.pi,100)
y = np.sin(x) + np.random.random(100) * 0.2
yhat = savgol_filter(score_history, 101, 7) # window size 51, polynomial order 3


plt.plot(yhat, color='red')
plt.show()

In [ ]:
print((avg_scores))